In [3]:
!pip install river

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.


In [1]:
import matplotlib.pyplot as plt
from river import (
    datasets,
    ensemble,
    metrics,
    tree
)

# --- 1. Настройка эксперимента ---

# Создаем синтетический поток данных SEA.
# variant=1 означает, что в середине потока произойдет резкое изменение
# в правилах классификации (внезапный дрейф).
drift_point = 2500
dataset = datasets.synth.SEA(seed=42, variant=1).take(5000)

# Список моделей, которые мы будем сравнивать
models = {
    "Одиночное дерево (HT)": tree.HoeffdingTreeClassifier(),

    "Обычный Бэггинг (Bagging)": ensemble.BaggingClassifier(
        model=tree.HoeffdingTreeClassifier(),
        n_models=10,
        seed=42
    ),

    # ИСПРАВЛЕНО: Используем 'LeveragingBaggingClassifier'.
    # Это мощный ансамбль, специально разработанный для работы с дрейфом.
    # Он сочетает бэггинг с механизмом ADWIN для повышения производительности.
    "Адаптивный Бэггинг (Leveraging Bagging)": ensemble.LeveragingBaggingClassifier(
        model=tree.HoeffdingTreeClassifier(),
        n_models=10,
        seed=42
    )
}

# Словарь для хранения метрик точности для каждой модели
metrics_dict = {name: metrics.Accuracy() for name in models.keys()}

# Словарь для хранения истории производительности для построения графика
performance_history = {name: [] for name in models.keys()}
plot_every_n_steps = 100

# --- 2. Цикл онлайн-обучения ---

print("Начинаем обработку потока данных...")

for i, (x, y) in enumerate(dataset):

    for model_name, model in models.items():

        y_pred = model.predict_one(x)

        if y_pred is not None:
            metrics_dict[model_name].update(y, y_pred)

        model.learn_one(x, y)

    if (i + 1) % plot_every_n_steps == 0:
        for model_name in models.keys():
            accuracy = metrics_dict[model_name].get()
            performance_history[model_name].append(accuracy)

print("Обработка завершена.")
print("-" * 30)

# --- 3. Вывод результатов ---

print("Итоговая точность:")
for model_name, metric in metrics_dict.items():
    print(f"  - {model_name}: {metric.get():.4f}")

fig, ax = plt.subplots(figsize=(12, 7))

x_axis = range(plot_every_n_steps, 5000 + 1, plot_every_n_steps)

for model_name, history in performance_history.items():
    ax.plot(x_axis, history, label=model_name, marker='o', markersize=3, alpha=0.8)

ax.axvline(drift_point, color='red', linestyle='--', label=f'Дрейф концепта (на {drift_point})')

ax.set_title("Сравнение производительности онлайн-ансамблей в условиях дрейфа концепта")
ax.set_xlabel("Количество обработанных примеров")
ax.set_ylabel("Точность (Prequential Accuracy)")
ax.legend()
ax.grid(True, linestyle='--', alpha=0.6)
ax.set_ylim(0.5, 1.05)

plt.show()

ModuleNotFoundError: No module named 'river'

Конечно! Вот подробное объяснение кода, специально отформатированное для текстовой ячейки в Jupyter Notebook. Вы можете просто скопировать и вставить этот текст в Markdown-ячейку над ячейкой с кодом.

---

# Анализ производительности онлайн-ансамблей в условиях дрейфа концепта

## 1. Цель эксперимента

Данный код представляет собой симуляцию, цель которой — наглядно продемонстрировать преимущества **адаптивных онлайн-ансамблей** по сравнению с одиночными моделями и простыми ансамблями при работе с нестационарными данными.

Ключевая проблема, которую мы исследуем, — это **дрейф концепта** (*concept drift*). Так называют ситуацию, когда статистические свойства данных или зависимости между ними со временем меняются. Модель, обученная на "старых" данных, начинает сильно ошибаться на "новых". Наша задача — сравнить, насколько быстро и эффективно разные модели могут адаптироваться к таким изменениям.

## 2. Дизайн эксперимента

### 2.1. Данные

Мы используем синтетический генератор потока данных `river.datasets.synth.SEA`. Его главное преимущество — возможность **контролируемого эксперимента**. Мы точно знаем, в какой момент времени (`drift_point = 2500`) в потоке данных произойдет резкое изменение правил, по которым метки присваиваются объектам. Это позволяет нам точно измерить реакцию моделей на дрейф.

### 2.2. Участники сравнения

Мы сравниваем три модели, представляющие три разных уровня сложности:

1.  **`tree.HoeffdingTreeClassifier` (Одиночное дерево)**
    *   **Роль:** Контрольная группа (baseline). Это эффективная базовая модель для потоковой классификации, но у нее нет специальных механизмов для борьбы с дрейфом. Мы ожидаем, что она будет медленнее всех восстанавливаться после изменений.

2.  **`ensemble.BaggingClassifier` (Обычный Бэггинг)**
    *   **Роль:** Простой ансамбль. Эта модель состоит из 10 деревьев Хоффдинга. Разнообразие моделей достигается за счет того, что каждое дерево обучается на своей случайной подвыборке данных (согласно распределению Пуассона). За счет усреднения предсказаний ансамбль должен быть более стабильным и точным, чем одиночная модель, но у него нет *активного* механизма адаптации к дрейфу.

3.  **`ensemble.LeveragingBaggingClassifier` (Адаптивный Бэггинг)**
    *   **Роль:** Продвинутый адаптивный ансамбль. Это усовершенствованная версия бэггинга. Она также состоит из 10 деревьев, но дополнительно использует механизм взвешивания на основе ошибок (ADWIN), чтобы повысить производительность в условиях дрейфа. Ансамбль отдает большее предпочтение тем моделям, которые лучше справляются с новыми данными, и быстрее избавляется от влияния устаревших знаний. Мы ожидаем, что эта модель покажет наилучшую скорость адаптации.

### 2.3. Процесс обучения и оценки

Мы симулируем **онлайн-обучение**, обрабатывая поток данных по одному объекту за раз. Для оценки качества используется протокол **Prequential-оценки** (или "Test-then-Train"):

1.  **Тест:** Для каждого нового объекта модель сначала делает предсказание.
2.  **Оценка:** Это предсказание сравнивается с истинной меткой, и метрика точности (Accuracy) обновляется.
3.  **Обучение:** И только после этого модель обучается на этом объекте.

Такой подход дает честную оценку способности модели работать с ранее невиданными данными.

## 3. Что происходит в коде?

*   **Блок 1 (Настройка):** Мы инициализируем генератор данных, создаем три наши модели и готовим структуры для сбора метрик и истории производительности.
*   **Блок 2 (Цикл онлайн-обучения):** Главный цикл, который итерируется по 5000 объектам из потока данных. Внутри цикла каждая из трех моделей проходит шаги "Тест -> Оценка -> Обучение". Каждые 100 шагов мы записываем текущее значение точности для последующей визуализации.
*   **Блок 3 (Результаты и визуализация):**
    *   Мы выводим итоговую точность каждой модели за весь прогон.
    *   Мы строим график, который является **главным результатом эксперимента**. На нем показано, как менялась скользящая точность каждой модели во времени. Красная пунктирная линия отмечает момент, когда в данных произошел дрейф.

## 4. Что ожидать на графике?

1.  **До дрейфа (до отметки 2500):** Все модели будут быстро обучаться, и их точность выйдет на высокий стабильный уровень (вероятно, выше 90%).
2.  **В момент дрейфа (на отметке 2500):** Мы увидим **резкое падение точности у всех моделей**. Это показывает, что их знания, полученные на старых данных, больше не актуальны.
3.  **После дрейфа (после отметки 2500):** Здесь проявится ключевое различие:
    *   Кривая **одиночного дерева** будет восстанавливаться очень медленно.
    *   Кривая **обычного бэггинга** будет подниматься быстрее за счет большей гибкости.
    *   Кривая **адаптивного бэггинга (`LeveragingBaggingClassifier`)** покажет самую высокую скорость восстановления. Благодаря своему внутреннему механизму адаптации, эта модель быстрее всех "поймет", что правила изменились, и перестроится под новые данные.

Таким образом, эксперимент наглядно демонстрирует, почему для работы с реальными, постоянно меняющимися данными, использование специализированных адаптивных ансамблей является критически важным.